In [11]:
import os
from os.path import isfile
import sys
import glob

os.environ["CUDA_VISIBLE_DEVICES"]="2"
#os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler

import mlflow
import mlflow.tensorflow

In [12]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("ATMS+CPL T")

<Experiment: artifact_location='file:///data/nature_run/work/src/mlruns/406530874024684188', creation_time=1727649901953, experiment_id='406530874024684188', last_update_time=1727649901953, lifecycle_stage='active', name='ATMS+CPL T', tags={}>

In [13]:
def get_data(filelist):
    hsel_val = []
    scalar_val = []
    table_val = []
    cpl_val = []

    for i in filelist:
        # Keys: 'mh', 'hsel', 'scalar', 'table']
        tmp = np.load(i)
        hsel_val.append(tmp["mh"])
        scalar_val.append(tmp["scalar"])
        table_val.append(tmp["table"])
        cpl_val.append(tmp["cpl"])

    return np.vstack(hsel_val), np.vstack(cpl_val), np.vstack(scalar_val), np.vstack(table_val)


train = ['/data/nature_run/fulldays_reduced/all_cpl_20060815.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20060915.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20061015.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20060515.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20060715.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20061115.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20060315.npz', 
         '/data/nature_run/fulldays_reduced/all_cpl_20061215.npz',
         '/data/nature_run/fulldays_reduced/all_cpl_20060615.npz']

test = ['/data/nature_run/fulldays_reduced/all_cpl_20060803.npz']

validation = ['/data/nature_run/fulldays_reduced/all_cpl_20060803.npz']

hsel_train, cpl_train, scalar_train, table_train = get_data(train)
hsel_test, cpl_test, scalar_test, table_test = get_data(test)
hsel_val, cpl_val, scalar_val, table_val = get_data(validation)



In [ ]:
#hsel_train, table_train, scalar_train, hsel_test, table_test, scalar_test = dataload("mh")

In [ ]:
#hsel_train, table_train, scalar_train, hsel_test, table_test, scalar_test = dataload("mh")
#hsel_val = hsel_test.copy()
#table_val = table_test.copy()
#scalar_val = scalar_test.copy()


In [14]:
print(hsel_train.shape)
print(cpl_train.shape)
print(table_train.shape)
print(scalar_train.shape)

(8414706, 22)
(8414706, 733)
(8414706, 72, 3)
(8414706, 44)


In [15]:
# What should we do
# 1) Only use train scalars
# 2) Scale seperateley


# Presssure ####
spress_test = scalar_test[:, 3].reshape(-1, 1)
spress_train = scalar_train[:, 3].reshape(-1, 1)
spress_val = scalar_val[:, 3].reshape(-1, 1)

mins = np.min(spress_train, axis=0)
maxs = np.max(spress_train, axis=0)
np.savez("spress_scalar_cpl.npz", mins=mins, maxs=maxs)

spress_test = (spress_test - mins)/(maxs - mins)
spress_train = (spress_train - mins)/(maxs - mins)
spress_val = (spress_val - mins)/(maxs - mins)


# Labels ####
q_train = table_train[:, :, 1]
q_test = table_test[:, :, 1]
q_val = table_val[:, :, 1]

# Spectra ####
mins = np.min(hsel_train, axis=0)
maxs = np.max(hsel_train, axis=0)

np.savez("minimac_scaling_factors_cpl_mh.npz", maxs=maxs, mins=mins)

hsel_train = np.nan_to_num(hsel_train)
hsel_test = np.nan_to_num(hsel_test)
hsel_val = np.nan_to_num(hsel_val)

hsel_train = (hsel_train - mins)/(maxs - mins)
hsel_test = (hsel_test - mins)/(maxs - mins)
hsel_val = (hsel_val - mins)/(maxs - mins)

# CPL #####
mins = np.min(cpl_train, axis=0)
maxs = np.max(cpl_train, axis=0)
np.savez("minimac_scaling_factors_cpl_cpl.npz", maxs=maxs, mins=mins)

cpl_train = (cpl_train - mins)/(maxs - mins)
cpl_test = (cpl_test - mins)/(maxs - mins)
cpl_val = (cpl_val - mins)/(maxs - mins)

with tf.device('/cpu:0'):
    # train
    x_train = {'rad': tf.convert_to_tensor(hsel_train, np.float32),
               'spress': tf.convert_to_tensor(spress_train, np.float32),
               'cpl': tf.convert_to_tensor(cpl_train, np.float32)}
    del hsel_train
    del spress_train
    
    y_train =  tf.convert_to_tensor(q_train, np.float32)
    del q_train

    
    # Val
    x_val = {'rad': tf.convert_to_tensor(hsel_val, np.float32),
             'spress': tf.convert_to_tensor(spress_val, np.float32),
             'cpl': tf.convert_to_tensor(cpl_val, np.float32)}
    del hsel_val
    del spress_val
    
    y_val =  tf.convert_to_tensor(q_val, np.float32)
    del q_val

    
    # Test
    x_test = {'rad': tf.convert_to_tensor(hsel_test, np.float32),
              'spress': tf.convert_to_tensor(spress_test, np.float32),
              'cpl': tf.convert_to_tensor(cpl_test, np.float32)}
    del hsel_test
    del spress_test
    
    y_test =  tf.convert_to_tensor(q_test, np.float32)
    del q_test


In [ ]:
# print(x_test)
# print(x_val)
# print(y_train.shape)
#a = np.argwhere(np.isnan(x_train['rad']))
#print(np.sum(np.isnan(x_train['rad'])))
#np.nan_to_num(x_train['rad'], copy=False)
#print(np.sum(np.isnan(x_train['rad'])))
#print(a)
#print(len(a))

np.sum(np.isnan(x_train['rad']))

In [16]:
def build_model(config):
    # ATMS 22
    mh = Input(shape=(config["shape"],), name="rad")
    spress = Input(shape=(1,), name="spress")
    # [ha, hb, hc, hd, hw, mh]
    
    x = Concatenate()([mh, spress])
    for i in range(config["num_layers"]):
        x =  Dropout(config["dropout"])(Dense(config["num_neurons"], 
                                             activation=config["activation"])(x))
    outputs = Dense(config['output'], name="Temp")(x)

    model = Model(inputs=[mh, spress], outputs=outputs, name="T")
    model.compile(optimizer="adam", loss='mae')

    return model

config = {'shape': 22,
          'output': 72,
          'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
          'num_layers': np.random.choice([3, 4, 5, 10]),
          'num_neurons': np.random.choice([16, 32, 64, 128]),
          'activation': np.random.choice(['gelu', 'gelu', 'relu'])}

model = build_model(config)
model.summary()


Model: "T"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 rad (InputLayer)               [(None, 22)]         0           []                               
                                                                                                  
 spress (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 concatenate_1 (Concatenate)    (None, 23)           0           ['rad[0][0]',                    
                                                                  'spress[0][0]']                 
                                                                                                  
 dense_12 (Dense)               (None, 32)           768         ['concatenate_1[0][0]']          

In [17]:
def build_cpl_model(config):
    # ATMS 22
    mh = Input(shape=(config["shape"],), name="rad")
    spress = Input(shape=(1,), name="spress")
    cpl = Input(shape=(733,), name="cpl")
    # [ha, hb, hc, hd, hw, mh]

    cpl_reduced = Dense(128, activation=config["activation"])(cpl)
    cpl_reduced = Dense(32, activation=config["activation"])(cpl_reduced)

    
    x = Concatenate()([mh, spress, cpl_reduced])
    for i in range(config["num_layers"]):
        x =  Dropout(config["dropout"])(Dense(config["num_neurons"], 
                                             activation=config["activation"])(x))
    outputs = Dense(config['output'], name="Temp")(x)

    model = Model(inputs=[mh, spress, cpl], outputs=outputs, name="T")
    model.compile(optimizer="adam", loss='mae')

    return model

config = {'shape': 22,
          'output': 72,
          'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
          'num_layers': np.random.choice([3, 4, 5, 10]),
          'num_neurons': np.random.choice([16, 32, 64, 128]),
          'activation': np.random.choice(['gelu', 'gelu', 'relu'])}

model = build_cpl_model(config)
model.summary()


Model: "T"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 cpl (InputLayer)               [(None, 733)]        0           []                               
                                                                                                  
 dense_22 (Dense)               (None, 128)          93952       ['cpl[0][0]']                    
                                                                                                  
 rad (InputLayer)               [(None, 22)]         0           []                               
                                                                                                  
 spress (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  

In [21]:
print("DING")

DING


In [ ]:
# cpl

for _ in range(100):
    config = {'shape': 22,
              'output': 72,
              'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
              'num_layers': np.random.choice([3, 4, 5, 10]),
              'num_neurons': np.random.choice([16, 32, 64, 128]),
              'activation': np.random.choice(['gelu', 'gelu', 'relu'])}

    model = build_cpl_model(config)
    
    with mlflow.start_run() as run:
        model_checkpoint_callback = ModelCheckpoint(
            filepath=f"models/model_{run.info.run_id}.keras",
            save_weights_only=False,
            monitor='val_loss',
            mode='min',
            save_best_only=True)
        callback = EarlyStopping(monitor='val_loss', patience=8, verbose=0)
        mlflow.log_param("CPL", True)

        history = model.fit(x_train, y_train, epochs=1000, batch_size=2000,
                            validation_data=(x_val, y_val),
                            verbose=2,
                            callbacks=[callback, model_checkpoint_callback])
        
        evaled_test = model.evaluate(x_train, y_train, batch_size=100, verbose=0)
        evaled_train = model.evaluate(x_test, y_test, batch_size=100, verbose=0)

        mlflow.log_metric("test_loss", evaled_test)
        mlflow.log_metric("train_loss", evaled_train)

        plt.plot(history.history["loss"])
        plt.plot(history.history["val_loss"])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plotname = f"plots/loss_plot_{run.info.run_id}.png"
        plt.savefig(plotname)
        plt.close()

        # Log the plot as an artifact
        mlflow.log_artifact(plotname)
        #model_path = "modles"
        #mlflow.tensorflow.log_model(tf_saved_model_dir=model_path, artifact_path="model")


2024/09/29 21:22:30 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/29 21:22:30 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


Epoch 1/1000
4208/4208 - 42s - loss: 10.4116 - val_loss: 11.0691 - 42s/epoch - 10ms/step
Epoch 2/1000
4208/4208 - 39s - loss: 5.7199 - val_loss: 12.9369 - 39s/epoch - 9ms/step
Epoch 3/1000
4208/4208 - 40s - loss: 5.5595 - val_loss: 11.1169 - 40s/epoch - 10ms/step
Epoch 4/1000
4208/4208 - 40s - loss: 5.4983 - val_loss: 11.3651 - 40s/epoch - 10ms/step
Epoch 5/1000
4208/4208 - 40s - loss: 5.4594 - val_loss: 10.9588 - 40s/epoch - 10ms/step
Epoch 6/1000
4208/4208 - 39s - loss: 5.4301 - val_loss: 11.8148 - 39s/epoch - 9ms/step
Epoch 7/1000
4208/4208 - 39s - loss: 5.4149 - val_loss: 12.2834 - 39s/epoch - 9ms/step
Epoch 8/1000
4208/4208 - 39s - loss: 5.3874 - val_loss: 11.8749 - 39s/epoch - 9ms/step
Epoch 9/1000
4208/4208 - 39s - loss: 5.3627 - val_loss: 10.7994 - 39s/epoch - 9ms/step
Epoch 10/1000
4208/4208 - 39s - loss: 5.3470 - val_loss: 11.1659 - 39s/epoch - 9ms/step
Epoch 11/1000
4208/4208 - 39s - loss: 4.7898 - val_loss: 10.8939 - 39s/epoch - 9ms/step
Epoch 12/1000
4208/4208 - 38s - loss

2024/09/29 21:43:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/data/jmackin1/miniconda/envs/tf-gpu/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2024/09/29 21:47:22 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/29 21:47:22 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


Epoch 1/1000
4208/4208 - 41s - loss: 41.3448 - val_loss: 125.7879 - 41s/epoch - 10ms/step
Epoch 2/1000
4208/4208 - 39s - loss: 29.4242 - val_loss: 39.6043 - 39s/epoch - 9ms/step
Epoch 3/1000
4208/4208 - 39s - loss: 25.7225 - val_loss: 20.7280 - 39s/epoch - 9ms/step
Epoch 4/1000
4208/4208 - 39s - loss: 24.2978 - val_loss: 9.1737 - 39s/epoch - 9ms/step
Epoch 5/1000
4208/4208 - 38s - loss: 23.2602 - val_loss: 4.9920 - 38s/epoch - 9ms/step
Epoch 6/1000
4208/4208 - 39s - loss: 22.6110 - val_loss: 5.1496 - 39s/epoch - 9ms/step
Epoch 7/1000
4208/4208 - 39s - loss: 22.2069 - val_loss: 5.3050 - 39s/epoch - 9ms/step
Epoch 8/1000
4208/4208 - 39s - loss: 21.8106 - val_loss: 4.7001 - 39s/epoch - 9ms/step
Epoch 9/1000
4208/4208 - 38s - loss: 21.4243 - val_loss: 4.7359 - 38s/epoch - 9ms/step
Epoch 10/1000
4208/4208 - 38s - loss: 21.0456 - val_loss: 4.7257 - 38s/epoch - 9ms/step
Epoch 11/1000
4208/4208 - 39s - loss: 20.6490 - val_loss: 4.7039 - 39s/epoch - 9ms/step
Epoch 12/1000
4208/4208 - 39s - loss

2024/09/29 23:17:13 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/29 23:17:13 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


...

Epoch 1/1000
4208/4208 - 41s - loss: 24.6550 - val_loss: 8.0882 - 41s/epoch - 10ms/step
Epoch 2/1000
4208/4208 - 38s - loss: 15.8935 - val_loss: 3.9703 - 38s/epoch - 9ms/step
Epoch 3/1000
4208/4208 - 39s - loss: 13.8491 - val_loss: 3.0976 - 39s/epoch - 9ms/step
Epoch 4/1000
4208/4208 - 38s - loss: 13.0556 - val_loss: 3.0951 - 38s/epoch - 9ms/step
Epoch 5/1000
4208/4208 - 37s - loss: 12.4896 - val_loss: 2.7971 - 37s/epoch - 9ms/step
Epoch 6/1000
4208/4208 - 37s - loss: 12.0847 - val_loss: 2.8779 - 37s/epoch - 9ms/step
Epoch 7/1000
4208/4208 - 38s - loss: 11.7845 - val_loss: 2.8778 - 38s/epoch - 9ms/step
Epoch 8/1000
4208/4208 - 38s - loss: 11.5586 - val_loss: 2.8710 - 38s/epoch - 9ms/step
Epoch 9/1000
4208/4208 - 38s - loss: 11.3512 - val_loss: 2.7714 - 38s/epoch - 9ms/step
Epoch 10/1000
4208/4208 - 38s - loss: 11.1655 - val_loss: 2.8436 - 38s/epoch - 9ms/step
Epoch 11/1000
4208/4208 - 38s - loss: 10.9835 - val_loss: 2.8349 - 38s/epoch - 9ms/step
Epoch 12/1000
4208/4208 - 37s - loss: 10

2024/09/30 06:28:23 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/30 06:28:23 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


Epoch 1/1000
4208/4208 - 40s - loss: 7.4986 - val_loss: 3.0148 - 40s/epoch - 9ms/step
Epoch 2/1000
4208/4208 - 38s - loss: 4.0810 - val_loss: 3.4843 - 38s/epoch - 9ms/step
Epoch 3/1000
4208/4208 - 38s - loss: 3.7877 - val_loss: 3.3383 - 38s/epoch - 9ms/step
Epoch 4/1000
4208/4208 - 38s - loss: 3.5061 - val_loss: 4.5109 - 38s/epoch - 9ms/step
Epoch 5/1000
4208/4208 - 38s - loss: 3.3906 - val_loss: 4.9762 - 38s/epoch - 9ms/step
Epoch 6/1000
4208/4208 - 38s - loss: 3.3228 - val_loss: 3.3544 - 38s/epoch - 9ms/step
Epoch 7/1000
4208/4208 - 38s - loss: 3.2572 - val_loss: 4.5194 - 38s/epoch - 9ms/step
Epoch 8/1000
4208/4208 - 38s - loss: 3.1983 - val_loss: 4.3113 - 38s/epoch - 9ms/step
Epoch 9/1000
4208/4208 - 38s - loss: 3.1484 - val_loss: 2.4357 - 38s/epoch - 9ms/step
Epoch 10/1000
4208/4208 - 38s - loss: 3.1132 - val_loss: 3.8424 - 38s/epoch - 9ms/step
Epoch 11/1000
4208/4208 - 38s - loss: 3.0668 - val_loss: 4.9576 - 38s/epoch - 9ms/step
Epoch 12/1000
4208/4208 - 38s - loss: 3.0301 - val_l

2024/09/30 06:43:21 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/30 06:43:21 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


Epoch 1/1000
4208/4208 - 40s - loss: 28.8886 - val_loss: 68.7798 - 40s/epoch - 10ms/step
Epoch 2/1000
4208/4208 - 38s - loss: 20.8481 - val_loss: 65.8887 - 38s/epoch - 9ms/step
Epoch 3/1000
4208/4208 - 38s - loss: 20.1994 - val_loss: 63.7533 - 38s/epoch - 9ms/step
Epoch 4/1000
4208/4208 - 38s - loss: 19.7568 - val_loss: 64.9549 - 38s/epoch - 9ms/step
Epoch 5/1000
4208/4208 - 38s - loss: 19.1383 - val_loss: 60.6394 - 38s/epoch - 9ms/step
Epoch 6/1000
4208/4208 - 38s - loss: 18.1576 - val_loss: 58.2761 - 38s/epoch - 9ms/step
Epoch 7/1000
4208/4208 - 39s - loss: 17.6739 - val_loss: 55.0985 - 39s/epoch - 9ms/step
Epoch 8/1000
4208/4208 - 38s - loss: 17.3285 - val_loss: 54.8693 - 38s/epoch - 9ms/step
Epoch 9/1000
4208/4208 - 38s - loss: 17.0108 - val_loss: 51.2649 - 38s/epoch - 9ms/step
Epoch 10/1000
4208/4208 - 39s - loss: 16.7024 - val_loss: 48.4408 - 39s/epoch - 9ms/step
Epoch 11/1000
4208/4208 - 38s - loss: 16.3116 - val_loss: 43.3789 - 38s/epoch - 9ms/step
Epoch 12/1000
4208/4208 - 38s

In [ ]:
# regs

for _ in range(100):
    config = {'shape': 22,
              'output': 72,
              'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
              'num_layers': np.random.choice([3, 4, 5, 10]),
              'num_neurons': np.random.choice([16, 32, 64, 128]),
              'activation': np.random.choice(['gelu', 'gelu', 'relu'])}
    
    model = build_model(config)
    
    with mlflow.start_run() as run:
        model_checkpoint_callback = ModelCheckpoint(
            filepath=f"models/model_{run.info.run_id}.keras",
            save_weights_only=False,
            monitor='val_loss',
            mode='min',
            save_best_only=True)
        callback = EarlyStopping(monitor='val_loss', patience=8, verbose=0)
        mlflow.log_param("CPL", False)

        history = model.fit(x_train, y_train, epochs=1000, batch_size=2000,
                            validation_data=(x_val, y_val),
                            verbose=2,
                            callbacks=[callback, model_checkpoint_callback])
        
        evaled_test = model.evaluate(x_train, y_train, batch_size=100, verbose=0)
        evaled_train = model.evaluate(x_test, y_test, batch_size=100, verbose=0)

        mlflow.log_metric("test_loss", evaled_test)
        mlflow.log_metric("train_loss", evaled_train)

        plt.plot(history.history["loss"])
        plt.plot(history.history["val_loss"])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plotname = f"plots/loss_plot_{run.info.run_id}.png"
        plt.savefig(plotname)
        plt.close()

        # Log the plot as an artifact
        mlflow.log_artifact(plotname)
        #model_path = "modles"
        #mlflow.tensorflow.log_model(tf_saved_model_dir=model_path, artifact_path="model")
